In [ ]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [ ]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [ ]:
data = pd.concat([pd.read_json(file, lines=True, encoding='UTF-8') for file in files], axis=0, ignore_index=True)

In [ ]:
data.shape

In [ ]:
data.head(3)

In [ ]:
data["keywords"].tolist()

In [ ]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

### Способ 1: использовать именительный падеж вместо нормальной формы, уменьшить min_df

In [ ]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.inflect({'nomn'}) for word in words if word.tag.POS == 'NOUN' or word.tag.POS == "ADJS" or word.tag.POS == "ADJF"]
    words = [word.word for word in words if word is not None]
    return words

In [ ]:
data['content_norm'] = data['content'].apply(normalize)

In [ ]:
data['content_norm2'] = data['content_norm'].apply(' '.join)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)

In [200]:
tfidf.fit(data['content_norm2'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm2'])
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
keywords[:3]

[['яблоко',
  'молодёжное яблоко',
  'молодёжное',
  'акция',
  'активисты',
  'молодёжный',
  'дарья',
  'наши активисты',
  'молодые люди',
  'деятельность'],
 ['млрд куб',
  'куб',
  'газпром',
  'газа',
  'млрд',
  'добыча газа',
  'добыча',
  'независимые производители',
  'холдинг',
  'производители'],
 ['роман',
  'мениппея',
  'книга',
  'жанр',
  'стихотворения',
  'поэзия',
  'книги',
  'перевод',
  'герои',
  'туманов страница']]

In [201]:
evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


### Сделаем граф направленным

In [ ]:
import networkx as nx
from itertools import combinations
import nltk

In [ ]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был направленный 
            m[j][k] += 1
    
    return m, id2word

In [ ]:
def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

In [198]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


### Как выяснилось, если в функцию nx.from_numpy_array(matrix) не задавать параметр create_using, независимо от матрицы (диагональную мы ей подаем или нет), в любом случае граф будет ненаправленный. Собственно, изменим код.

In [ ]:
def x_build_matrix(text, window_size):
    return build_matrix(text, window_size)

In [ ]:
data['matrix'] = data['content_norm'].apply(lambda x: x_build_matrix(x, 10)[0])

In [ ]:
data['id2word'] = data['content_norm'].apply(lambda x: x_build_matrix(x, 10)[1])

In [ ]:
def compute_kws(matrix):
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
    return dict(nx.degree(G))

In [ ]:
measures = data['matrix'].apply(lambda x: compute_kws(x))

In [ ]:
def return_ids(id2word, node2measure, topn):
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
directed_kws = [return_ids(data['id2word'][i], measures[i], 10) for i in range(len(measures))]

In [197]:
evaluate(data['keywords'], directed_kws)

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


Давайте проверим, а изменилось ли что-то в результатах (потому что при предыдущем запуске эксперимента результаты только переранжировались)

In [196]:
for i in range(10):
    print(set.difference(set(directed_kws[i]), set(keyword_nx[i])), set.difference(set(keyword_nx[i]), set(directed_kws[i])))

set() set()
set() set()
{'стихотворения'} {'мениппея'}
{'бывший'} {'рубли'}
{'развитие'} {'новые'}
{'ситуация'} {'который'}
set() set()
{'вопрос'} {'новый'}
{'иран'} {'газовые'}
set() set()


### Мы видим, что разница все-таки есть. Но не зря же я дробила код, чтобы потом не попробовать другие метрики? Давайте посмотрим на closeness centrality, betweenness centrality и eigenvector centrality

In [242]:
def compute_cc_kws(matrix):
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
    return dict(nx.closeness_centrality(G))

In [243]:
def compute_bc_kws(matrix):
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
    return dict(nx.betweenness_centrality(G))

In [249]:
def compute_ec_kws(matrix):
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
    return dict(nx.eigenvector_centrality(G))

In [245]:
measures = data['matrix'].apply(lambda x: compute_cc_kws(x))

In [246]:
cc_kws = [return_ids(data['id2word'][i], measures[i], 10) for i in range(len(measures))]
evaluate(data['keywords'], cc_kws)

Precision -  0.1
Recall -  0.2
F1 -  0.13
Jaccard -  0.07


In [247]:
measures = data['matrix'].apply(lambda x: compute_bc_kws(x))
bc_kws = [return_ids(data['id2word'][i], measures[i], 10) for i in range(len(measures))]
evaluate(data['keywords'], bc_kws)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


### Оно, конечно, замечательно, но давайте попробуем что-то интересненькое

In [ ]:
import RAKE

In [ ]:
def anti_normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word]
    words = [word.word for word in words if word.tag.POS != 'NOUN' and word.tag.POS != "ADJS" and word.tag.POS != "ADJF" and word is not None]
    return words

In [ ]:
data['stops'] = data['content'].apply(lambda x: anti_normalize(x))

In [ ]:
data['rakes'] = data['stops'].apply(lambda x: RAKE.Rake(list(set(x))))

In [ ]:
rake_kws = []
for i in range(data.shape[0]):
    rake_kws.append(data["rakes"].tolist()[i].run(data['content'].tolist()[i]))

In [ ]:
rake_kws_0 = [[i[0] for i in row][:10] for row in rake_kws]

In [195]:
evaluate(data['keywords'], rake_kws_0)

Precision -  0.0
Recall -  0.0
F1 -  0.0
Jaccard -  0.0


Как-то не очень. попробуем как-то преобразовать результат

In [225]:
r = [normalize(' '.join([i[0] for i in r][:20])) for r in rake_kws]

In [229]:
r_joined = [' '.join(i) for i in r]

Посмотрим на TFIDF

In [235]:
tfidf_rake = TfidfVectorizer(ngram_range=(1,2), min_df=1)
tfidf_rake.fit(r_joined)
id2word_rake = {i:word for i,word in enumerate(tfidf_rake.get_feature_names())}
texts_vectors_rake = tfidf_rake.transform(r_joined)
# сортировка по убыванию, поэтому нужно развернуть список
keywords_rake = [[id2word_rake[w] for w in top] for top in texts_vectors_rake.toarray().argsort()[:,:-11:-1]] 
keywords_rake[:3]

[['яблоко',
  'молодёжное яблоко',
  'молодёжное',
  'молодые люди',
  'молодые',
  'молодёжный яблоко',
  'задача молодёжный',
  'кампания страна',
  'арбатов нынешние',
  'полные жизнь'],
 ['добыча газа',
  'газа',
  'цены',
  'добыча',
  'холдинг',
  'технологии очередные',
  'михаил занозины',
  'суммарная добыча',
  'обеспечение внутреннее',
  'сильный козырь'],
 ['моя',
  'наследник',
  'александр',
  'прозаик прямое',
  'новеллы',
  'евгений витковский',
  'маркес',
  'литература законный',
  'маркес конец',
  'памятные эти']]

In [236]:
evaluate(data['keywords'], keywords_rake)

Precision -  0.05
Recall -  0.1
F1 -  0.06
Jaccard -  0.04


Посмотрим на textRank

In [240]:
kws_rake_rank = [some_centrality_measure(r_, 10, 10) for r_ in r]
evaluate(data['keywords'], kws_rake_rank)

Precision -  0.05
Recall -  0.11
F1 -  0.07
Jaccard -  0.04


In [241]:
kws_rake_rank[0]

['политика',
 'люди',
 'яблоко',
 'молодые',
 'ильдар',
 'молодёжная',
 'арбатов',
 'опасная',
 'страна',
 'условия']

Ну, всяко лучше, чем ничего..

### Попробуем TextRank

In [ ]:
from summa import keywords

In [ ]:
textrank = data['content'].apply(lambda x: keywords.keywords(x))

In [ ]:
evaluate(data['keywords'], textrank)

In [ ]:
textrank_norm = data['content_norm2'].apply(lambda x: keywords.keywords(x))
evaluate(data['keywords'], textrank_norm)

In [150]:
list_tx_kws = []
for i in data['content_norm2'].tolist():
    list_tx_kws.append(keywords.keywords(i).split('\n')[:10])

In [151]:
evaluate(data['keywords'], list_tx_kws)

Precision -  0.07
Recall -  0.13
F1 -  0.09
Jaccard -  0.05


Получилось грустненько. Ну ничего, попробуем что-нибудь еще

### Попробуем викификацию от моей любимой Texterra. Вдруг она не подведет

In [155]:
import texterra
t = texterra.API("c41d9b98960e6f6bdfb3452f6b174e5a6554f992")

Так как текстерра использует API, то, чтобы не DDOS-ить сервер, сделаем поменьше запросов

In [180]:
texterra_kws = data['content_norm2'][:10].apply(lambda x: list(t.disambiguation(x)))

In [186]:
texterra_kws2 = data['content_norm2'][20:50].apply(lambda x: list(t.disambiguation(x)))

In [181]:
t_kws = [list(set([t[2] for t in kws[0]])) for kws in texterra_kws]

In [182]:
evaluate(data['keywords'][:10], t_kws)

Precision -  0.06
Recall -  0.45
F1 -  0.1
Jaccard -  0.05


In [193]:
t_kws2 = [list(set([t[2] for t in kws[0]])) for kws in texterra_kws2]

In [194]:
evaluate(data['keywords'].tolist()[20:50], t_kws2)

Precision -  0.06
Recall -  0.38
F1 -  0.1
Jaccard -  0.05


Как мы видим, у нас приличный recall, а вот с precision беда.
Так как результат не превосходит baseline уже сейчас, то мучить сервер и запускать на остальных данных, думаю, не стоит

In [211]:
texterra_kws_counter = [Counter(i).most_common(10) for i in t_kws2]
texterra_kws_counter = [[j[0] for j in i] for i in texterra_kws_counter]

In [213]:
evaluate(data['keywords'].tolist()[20:50], texterra_kws_counter)

Precision -  0.07
Recall -  0.12
F1 -  0.09
Jaccard -  0.05
